# 1 Introducción

Para este ejercicio se ha optado por aplicar el tema teórico **MPI** (message passing interface).

La finalidad de este ejercicio será ampliar el conocimiento sobras la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada **MPI4py**.

Para esto, se optará por un desarrollo que utilizará un patrón Master-Slave en donde un proceso se comunique y envíe órdenes a los demás.

En primera instancia, se debe ejecutar el armado del ambiente para instalar la libreria, luego se debe ejecutar el codigo el cual generara un codigo Python en un archivo, por ultimo se debe ejecutar el bloque de codigo en la subseccion 3.1 para poder ver los resultados.

#2 Armado del ambiente

Instalar en el cuaderno el módulo MPI4py de Python.

In [ ]:
! pip install mpi4py

#3 Desarrollo

In [ ]:
%%writefile Ejercicio3.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   5#@param {type: "number"}
Min_tiempo_sleep =   1#@param {type: "number"}
# --------------------------------------------

# --------------------------------------------
# Valido número de ingreso por el usuario
if (Max_tiempo_sleep <= 0 or Min_tiempo_sleep <= 0 ):
  raise Exception("Por favor, ingrese números positivos.") 
# -------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------


def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank() # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init() # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size()  # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4 # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks) # Se generan las tareas, en este caso seran 
        print("Master crea {} valores para el sleep de los slaves:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

# Funcion que tendra lugar 1 sola vez al crearse el Master.
def init():
  print()
  print( "------------------------------------", flush=True)
  print( "Informacion General:", flush=True)
  print()
  # Imprimimos la version de la bibliotica MPI4py
  print ("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  # Imprimimos el procesador utilizado
  print ("Procesador: {}".format(MPI.Get_processor_name()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print()


def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0
    print("Master enviando las tareas:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Master envia {} al slave: {}".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los slaves y se sigue enviando trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status() 
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los slaves.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el numero de slave.
        print("Master recibe {} del slave: {}".format(workTime, workerId), flush=True)
        #send next work
        comm.send(workTimes[workcount], dest=workerId, tag=WORK_FLAG) # Funcion bloqueante, comparte un buffer de memoria entre el llamador y el llamado, el llamador no continua su ejecucion hasta que este buffer sea liberado. 
        workcount += 1
        print("Master envia {} al slave: {}".format(work, workerId), flush=True)

    # Recivir los resultados de las peticiones de trabajo.
    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Fin: Master recibe {} del slave: {}".format(workTime, workerId), flush=True)

    # Parar a todos los slaves.
    for id in range(1, numProcesses):
        comm.send(-1, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    # El slave realizara sus tareas hasta que reciba un mensaje de finalizar desde el Master.
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del slave.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Master.
        print("Slave {} recibe {}".format(comm.Get_rank(), waitTime), flush=True)
        # Si el Master envia que debe finalizar, entonces no se realiza ninguna tarea y retorna.
        if (stat.Get_tag() == END_WORK_FLAG):
            print("Slave: {} finaliza su trabajo.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime) # Realiza el trabajo
        comm.send(waitTime, dest=0) # Envia el fin del trabajo al Master

def generateTasks(numTasks):
    np.random.seed(1000)  # Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

# Ejecutar la funcion principal.
main()


## 3.1 Ejecutar prueba del desarollo

In [ ]:
! mpirun --allow-run-as-root -np 4 python Ejercicio3.py

#4 Tabla de pasos de ejecución del programa


 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tiempo máximo que debe dormir un slave.
CPU      |  @param                | Lectura del tiempo mínimo que debe dormir un slave.
CPU      |  constantes            | Bloque de definición de constantes.
CPU      |  Main - MPI.COMM_WORLD        | Inicializa el comunicador entre procesos.
CPU      |  Main - comm.Get_rank()      | Obtiene el proceso actual que se ejecuta.
CPU      |  Main - init()             | Muestra información general.
CPU      |  Main - comm.Get_size()    | Obtiene la cantidad de procesos totales.
CPU      |  Main - generateTasks(numTasks)       | Genera las tareas en un array.
CPU      |  Main - initWork(comm, workTimes, numProcesses) | El Master inicia las peticiones de trabajo.
CPU      |  Main - doWork(comm)         | Los slaves tienen asignada la funcion para su trabajo.
CPU      |  initWork - comm.send(work, dest=id, tag=WORK_FLAG)            | El Master envía el trabajo al slave.
CPU      | doWork - comm.recv(source=0, tag=MPI.ANY_TAG, status=stat)  | El Slave recibe el trabajo del Master.
CPU      | doWork - comm.send(waitTime, dest=0)                        | El Slave le envia la respuesta de trabajo realizado al Master.
CPU      |  initWork - comm.recv(source=MPI.ANY_SOURCE, status=stat)   | El Master recibe el mensaje de trabajo realizado por el slave.
CPU      |  initWork - comm.send(-1, dest=id, tag=END_WORK_FLAG)               | El Master envía el mensaje para informar a los slaves del fin del trabajo.


#5 Conclusiones

La librería MPI4py provee una interfaz sencilla y práctica para la gestión de tareas concurrentes en distintos procesos sin la necesidad de codificar distintos hilos y realizar una sincronización entre ellos.

La primer duda que tuve al empezar con la codificación del ejercicio fue en que momento se instancia cada hilo, esto sucede directamente al momento donde importamos la librería o, si es deseado, se pueden invocar funciones como **MPI.Init()** o **MPI.Init_thread()** para hacerlo de manera más manual.

Esta librería provee una amplia variedad de funciones con variedad de usos y la consideré especialmente útil a la hora de generar un patrón Master-Slave debido a que para el Master es posible comunicarse con uno solo de los Slave o en modo broadcast con todos ellos de manera sincroniza o asincrónica.

Debido al límite de tiempo no he podido explorar la totalidad de esta librería, no obstante, este ejercicio posee muchas formas de ser ampliado o mejorado. Por ejemplo, es posible implementar un modelo productor consumidor de manera sencilla o realizar algunas tareas de manera asincrónica y otras sincronizadas.


#6 Referencias



*   [1] https://mpi4py.readthedocs.io/en/stable/

